<a href="https://colab.research.google.com/github/chathumal93/ADB-EARR-T4/blob/main/ALOS2_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install rasterio
!pip install pyproj

In [24]:
import os
import glob
from zipfile import ZipFile

import rasterio

from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from rasterio.warp import calculate_default_transform, reproject, Resampling,transform_bounds

import numpy as np

from scipy.signal import medfilt2d

In [3]:
image_directory = "/content/drive/MyDrive/Flood_Data_Myanmar/Thai_Data"

# image_path = "/content/drive/MyDrive/**/**.zip"

image_path = "/content/drive/MyDrive/Flood_Data_Myanmar/Thai_Data/16c19b23-9422-4316-a261-d70575dbee32.zip"

output_directory = "/content/Results"



temp_slope = ""

temp_dem = ""


In [8]:
if not os.path.exists(output_directory):   
  os.makedirs(output_directory)
with ZipFile(image_path, 'r') as zipObj:
  listOfFileNames = zipObj.namelist()
  for file_name in listOfFileNames:
    if file_name.endswith('.tif'):  
      zipObj.extract(file_name,output_directory)

In [18]:
#@markdown Helper Functions

def utm_finder(raster):

  """
  Find the UTM EPSG code of a raster.
  Arguments:  
        raster: input raster path
  Returns:
        UTM EPSG code of the input raster
  """

  with rasterio.open(raster) as dataset:
    bbox  = dataset.bounds
    bbox_wgs84 = rasterio.warp.transform_bounds(dataset.crs,'EPSG:4326', bbox[0],bbox[1],bbox[2],bbox[3])

    utm_crs_list = query_utm_crs_info(     
        datum_name="WGS 84",
        area_of_interest=AreaOfInterest(
        west_lon_degree=bbox_wgs84[0],
        south_lat_degree=bbox_wgs84[1],
        east_lon_degree=bbox_wgs84[2],
        north_lat_degree=bbox_wgs84[3],),) 
       
    utm_crs = "{}:{}".format(utm_crs_list[0].auth_name,utm_crs_list[0].code) 
  return utm_crs


def resample(raster,pixel_size,output_dir,output_name):

  """
  """

  with rasterio.open(raster) as dataset:
    kwargs = dataset.meta.copy()
    # resample data to target shape
    upscale_factor = dataset.transform[0]/pixel_size
    
    data = dataset.read(out_shape=(dataset.count,int(dataset.height * upscale_factor),int(dataset.width * upscale_factor)),resampling=Resampling.bilinear)
    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2]))
    
    # writing file to disk 
    kwargs.update({'crs': dst_crs,'transform': transform,'width': data.shape[2],'height': data.shape[1]})
    with rasterio.open(os.path.join(output_dir,output_name), 'w', **kwargs) as dst:
      dst.write(data[0], 1)





In [20]:
file_list = glob.glob(os.path.join(output_directory,'*.tif'))

file_list[0]

'/content/Results/IMG-HV-ALOS2453343350-221015-WBDR2.1GUD.tif'

In [7]:
dst_crs = utm_finder(file_list[0])

In [36]:
for file_path in glob.glob(os.path.join(output_directory,'*.tif')):

  if os.path.basename(file_path).startswith('IMG-HV'):
    print(os.path.basename(file_path).split(".tif")[0])
    print(utm_finder(file_path))

  if os.path.basename(file_path).startswith('IMG-HH'):
    print(os.path.basename(file_path).split(".tif")[0])
    print(utm_finder(file_path))
    
  else:
    pass


IMG-HV-ALOS2453343350-221015-WBDR2.1GUD
EPSG:32647
IMG-HH-ALOS2453343350-221015-WBDR2.1GUD
EPSG:32647


In [22]:
resample(file_list[0],30,output_directory,'resample.tif')

In [25]:
def calibration(raster,output_dir,output_name):

  """

  """

  with rasterio.open(raster) as dataset:

    # tranformation calculation for UTM reprojection
    transform, width, height = calculate_default_transform(dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)
    kwargs = dataset.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height,
        'compress': 'lzw',
        'dtype':'float32',
        'nodata':np.nan})
    
    # Calibration
    band = dataset.read(1)
    band = np.float32(np.where(band == 0, np.nan, band))
    band_calib = 20*np.log10(band)-83

    # Filtering
    med =  medfilt2d(band_calib, kernel_size=3)
                  
    with rasterio.open(os.path.join(output_dir,output_name), "w", **kwargs) as dst:
      for i in range(1, dataset.count + 1):
          reproject(
              source=med,
              destination=rasterio.band(dst, i),
              src_transform=dataset.transform,
              src_crs=dataset.crs,
              dst_transform=transform,
              dst_crs=utm_finder(raster),
              resampling=Resampling.nearest)


In [27]:
calibration('/content/Results/resample.tif',image_directory,'calib_reporj.tif')

In [ ]:
# 30m resolution